In [1]:
!pip install parser-2gis


[notice] A new release of pip is available: 24.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install openpyxl


[notice] A new release of pip is available: 24.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import json

# Инициализация драйвера Chrome
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Открываем страницу с полем ввода адреса
driver.get('https://platform.2gis.ru/ru/playground/geocoder')

# Функция для получения данных по адресу

def get_address_data(address, city):
    try:
        full_query = f"{address}, {city}"
        
        # Открыть страницу
        driver.get('https://platform.2gis.ru/ru/playground/geocoder')
        print(f"Страница загружена для запроса: {full_query}")
        
        # Найти поле ввода и ввести полный адрес (адрес + город)
        search_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-n="address-search-input"]'))
        )
        search_input.clear()
        search_input.send_keys(full_query)
        print(f"Введён запрос: {full_query}")

        # Подождать и выбрать первую подсказку
        suggestion = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-n="items-button"]'))
        )
        suggestion.click()
        print(f"Подсказка выбрана для запроса: {full_query}")

        # Нажать на кнопку "Ответ"
        response_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-n="response-tab"]'))
        )
        response_button.click()
        print(f"Нажата кнопка 'Ответ' для запроса: {full_query}")

        # Ожидание загрузки ответа и получение текста ответа
        response_area = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '[data-n="response-code-area"] pre'))
        )
        response_text = response_area.text
        print(f"Ответ для запроса {full_query}: {response_text}")
        
        # Преобразование ответа из JSON
        response_json = json.loads(response_text)
        
        # Извлечение всех полей
        item = response_json['result']['items'][0]
        full_data = {
            'address_name': item.get('address_name', ''),
            'building_name': item.get('building_name', ''),
            'full_name': item.get('full_name', ''),
            'id': item.get('id', ''),
            'name': item.get('name', ''),
            'lat': item['point'].get('lat', ''),
            'lon': item['point'].get('lon', ''),
            'purpose_name': item.get('purpose_name', ''),
            'region_id': item.get('region_id', ''),
            'segment_id': item.get('segment_id', ''),
            'type': item.get('type', '')
        }
        
        return full_data

    except Exception as e:
        print(f"Ошибка при обработке запроса: {full_query}, Error: {str(e)}")
        return None

# Загрузка файла с адресами
file_path = 'Кокшетау_бц.xlsx'
df = pd.read_excel(file_path)

# Проверка наличия необходимых колонок
if 'Адрес' not in df.columns or 'Город' not in df.columns:
    raise ValueError("Файл должен содержать колонки 'Адрес' и 'Город'")

# Создание новых колонок в DataFrame для данных API
df[['address_name', 'building_name', 'full_name', 'id', 'name', 'lat', 'lon', 'purpose_name', 'region_id', 'segment_id', 'type']] = ''

# Проход по каждому адресу и заполнение данными
for index, row in df.iterrows():
    address = row["Адрес"]
    city = row["Город"]
    full_data = get_address_data(address, city)
    
    if full_data:
        for key in full_data:
            df.at[index, key] = full_data[key]

# Сохранение результата в Excel
df.to_excel(file_path, index=False)

# Закрытие браузера
driver.quit()


Страница загружена для запроса: улица Толеу Сулейменова, 23/5, Кокшетау
Введён запрос: улица Толеу Сулейменова, 23/5, Кокшетау
Подсказка выбрана для запроса: улица Толеу Сулейменова, 23/5, Кокшетау
Нажата кнопка 'Ответ' для запроса: улица Толеу Сулейменова, 23/5, Кокшетау
Ответ для запроса улица Толеу Сулейменова, 23/5, Кокшетау: {
  "meta": {
    "api_version": "3.0.18728",
    "code": 200,
    "issue_date": "20250408"
  },
  "result": {
    "items": [
      {
        "address_name": "улица Толеу Сулейменова, 23/5",
        "building_name": "Silk Way, бизнес-центр",
        "full_name": "Кокшетау, Silk Way, бизнес-центр",
        "id": "70030076254221905",
        "name": "Silk Way, бизнес-центр",
        "point": {
          "lat": 53.306551,
          "lon": 69.396552
        },
        "purpose_name": "Бизнес-центр",
        "region_id": "201",
        "segment_id": "0",
        "type": "building"
      }
    ],
    "total": 1
  }
}
Страница загружена для запроса: улица Абая Кунанб

In [5]:
import os
import pandas as pd
import random
import time
import shutil
import subprocess

# 🔹 Загружаем данные
file_path = r"C:\Users\zhara\Documents\Кокшетау_бц.xlsx"
print(f"Загрузка данных из {file_path}...")
df = pd.read_excel(file_path, dtype={"id": str})  # Указываем, что столбец "id" должен быть строкой

# 🔹 Исправление формата ID
if "id" in df.columns:
    df["id"] = df["id"].astype(str).str.strip()

# 🔹 Генерация URL для парсинга
base_url = "https://2gis.kz/kokshetau/inside/"
df["url"] = base_url + df["id"]

# 🔹 Уникальные URL
unique_urls = df["url"].dropna().unique()
print(f"Найдено {len(unique_urls)} уникальных URL.")

# 🔹 Директория для сохранения файлов
output_dir = r"C:\Users\zhara\Documents\Кокшетау_бц.xlsx"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)
print(f"Каталог очищен и создан: {output_dir}")

# 🔹 Функция для смены User-Agent
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
]

def get_random_headers():
    return {"User-Agent": random.choice(user_agents)}

# 🔹 Запуск парсинга
for i, url in enumerate(unique_urls):
    place_id = url.split("/")[-1]
    output_path = os.path.join(output_dir, f"{place_id}.csv")

    # Команда для парсера
    command = f'parser-2gis -i "{url}" -o "{output_path}" -f csv --writer.csv.add-rubrics yes --writer.csv.remove-duplicates yes --writer.encoding utf-8-sig --parser.max-records 1000 --writer.verbose yes'

    print(f"\n--- Выполняется {i+1}/{len(unique_urls)}: {command} ---\n")
    
    # Используем subprocess для получения ошибок
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    if result.returncode != 0:
        print(f"❌ Ошибка выполнения команды:\n{result.stderr}")
        continue  # Пропускаем итерацию, если парсинг не удался

    # Проверяем, создался ли файл
    if os.path.exists(output_path):
        df_parsed = pd.read_csv(output_path, encoding='utf-8-sig')

        # Добавляем проверку содержимого
        print(f"\n📂 Проверка содержимого {output_path}:")
        print(df_parsed.head(10))

        if df_parsed.shape[0] == 0:
            print(f"⚠️ Внимание! В файле {output_path} нет данных, кроме заголовков.")
        else:
            print(f"✅ Файл {output_path} успешно загружен с {df_parsed.shape[0]} записями.")
    else:
        print(f"❌ Ошибка: Файл {output_path} не найден!")

    # Рандомная задержка перед следующим запросом
    time.sleep(random.uniform(3, 8))

# 🔹 Объединение всех CSV-файлов в один
print("🔍 Проверка файлов в output_dir...")
all_csv_files = [os.path.join(output_dir, f) for f in os.listdir(output_dir) if f.endswith(".csv")]
print(f"📂 Всего загружено файлов: {len(all_csv_files)}")

if all_csv_files:
    print("🔗 Объединение всех CSV-файлов...")

    # Список для хранения DataFrame'ов
    merged_data = []

    for file in all_csv_files:
        df_temp = pd.read_csv(file, encoding='utf-8-sig')
        df_temp["filename"] = os.path.basename(file)  # Добавляем название файла в столбец "filename"
        merged_data.append(df_temp)

    # Объединяем все файлы в один DataFrame
    merged_df = pd.concat(merged_data, ignore_index=True)

    # Сохраняем итоговый файл
    merged_file_path = os.path.join(output_dir, "Кокшетау_бц.csv")
    merged_df.to_csv(merged_file_path, index=False, encoding='utf-8-sig')

    print(f"✅ Объединенный файл сохранен: {merged_file_path}")

    # Отображаем часть данных в Jupyter Notebook
    display(merged_df.head(10))
else:
    print("⚠️ Не найдено CSV-файлов для объединения.")

Загрузка данных из C:\Users\zhara\Documents\Кокшетау_бц.xlsx...
Найдено 12 уникальных URL.


NotADirectoryError: [WinError 267] Неверно задано имя папки: 'C:\\Users\\zhara\\Documents\\Кокшетау_бц.xlsx'